In [ ]:
pip install pandas

In [4]:
import pandas as pd

In [6]:
nodes = pd.read_csv("nodes.tsv", sep="\t")
edges = pd.read_csv("edges.tsv", sep="\t")

In [7]:
nodes.head()

,osmid,x,y,primary
0,12885866,-21.937473,64.067271,False
1,12885876,-21.939951,64.076157,True
2,12885922,-22.087890,64.031977,False
3,12885923,-21.937852,64.067260,False
4,12885924,-21.938078,64.067047,True


In [8]:
edges.head()

,u,v,length,name
0,12885866,12885923,18.992015,Hlíðartorg
1,12885866,1573411414,653.666145,Reykjanesbraut
2,12885876,1660480490,35.437064,Flatahraun
3,12885876,12885952,16.131272,FH-Torg
4,12885922,2978379988,2587.067864,Reykjanesbraut
